### Initialization

In [1]:
# try:
#     import nvtabular as nvt
# except:
#     !pip install merlin-core==0.3.0 nvtabular==1.1.1 merlin-models==0.4.0 merlin-systems==0.2.0
# import nvtabular as nvt
# from merlin.core.dispatch import get_lib
# df_lib = get_lib()
# df_lib

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm.notebook import tqdm
from collections import Counter
from joblib import Parallel, delayed

### Data

In [3]:
DATA_PATH = Path('../input/otto-recommender-system')

TRAIN_PATH = DATA_PATH / 'train.jsonl'
TEST_PATH = DATA_PATH / 'test.jsonl'

CLASSES = ['clicks', 'carts', 'orders']

In [4]:
OUT_DIR = "processed/"
os.makedirs(OUT_DIR, exist_ok=True)

sample_size = 10000
n_chunks = 12899779 // sample_size + 1

chunks = pd.read_json(TRAIN_PATH, lines=True, chunksize=sample_size)

In [5]:
def process_data(chunk, save_folder=""):
    arrays = c['events'].apply(lambda x: np.array([[c['aid'], c['ts'], CLASSES.index(c['type'])] for c in x]).astype(int))

    if save_folder:
        for session, array in zip(chunk['session'], arrays):
            np.save(save_folder + f"session_{session}.npy", array)

    return arrays

In [6]:
%%time

for c in chunks:
    arrays = process_data(c, "/tmp/")
    break

CPU times: user 1.8 s, sys: 1.41 s, total: 3.21 s
Wall time: 3.6 s


In [7]:
len(arrays), arrays[0].shape

(10000, (276, 3))

In [8]:
for c in tqdm(chunks, total=n_chunks):
    process_data(c, OUT_DIR)
    
    break

  0%|          | 0/1290 [00:00<?, ?it/s]

In [9]:
# chunks = pd.read_json(TRAIN_PATH, lines=True, chunksize=sample_size)

# _ = Parallel(n_jobs=4)(
#     delayed(process_data)(c, OUT_DIR)
#     for c in tqdm(chunks, total=n_chunks)
# )

### NVT

In [10]:
# train_dataset = nvt.Dataset(PQ_DIR, engine="parquet")

In [11]:
# group = ['aid', 'session'] >> nvt.ops.Groupby(
#     groupby_cols=['session'],
#     aggs={'aid': 'list'},
# )

In [12]:
# clicks = ["type"] >> nvt.ops.LambdaOp(lambda x: x == "clicks")
# carts = ["type"] >> nvt.ops.LambdaOp(lambda x: x == "carts")
# orders = ["type"] >> nvt.ops.LambdaOp(lambda x: x == "orders")

# output = group + clicks # + carts + orders + group

# (output).graph

In [13]:
# workflow = nvt.Workflow(output)

In [14]:
# %%time
# workflow.fit(train_dataset)

In [15]:
# workflow.transform(train_dataset).to_parquet("test.parquet")